In [2]:

import sympy as sp

from sympy import Symbol
from sympy import symbols
from sympy import Function
from sympy import diff
from sympy import I
from sympy import exp
from sympy import solve
from sympy import Eq
from sympy import Poly
from sympy import sqrt
from sympy import tanh
from sympy import coth
from sympy import fraction

from IPython import embed


def print_hello_msg():
	print("[main]: mephim/eq\n")
	pass



In [3]:

def f_extract_from_eq_sys(eq_sys, param, eq_num, flag_return_new_eq_sys = True):
	if isinstance(param, str):
		param = eval(param)
	param_s = solve(eq_sys[eq_num], param)[0]
	if flag_return_new_eq_sys == True:
		new_eq_sys = [ eq.subs({ param : param_s }).doit() for eq in eq_sys ]
		return param_s, new_eq_sys
	return param_s, eq_sys


def f_extract_from_eq_sys_(eq_sys, param_list, eq_num_list):
	if eq_num_list == []:
		eq_num_list = list(range(0, len(eq_sys)))
	param_res_d = { }
	new_eq_sys = eq_sys
	for index, param in enumerate(param_list):
		param_s_, new_eq_sys = f_extract_from_eq_sys(new_eq_sys, param, eq_num_list[index])
		param_res_d[param] = param_s_
	return param_res_d



print_hello_msg()


[main]: mephim/eq



In [4]:

def f_gen_target_eq(n):
	coeff_list = { }
	q_der_list = [ ]
	q = Function("q", nargs = 2)
	y = Function("y", nargs = 1)
	x = Symbol("x", real = True)
	t = Symbol("t", real = True)
	for i in range(1, n * 2 + 1):
		key = "a{}".format(i)
		value = eval("Symbol(\"{}\")".format(key))
		coeff_list[key] = value
	qt = diff(q(x, t), t, 1)
	q_der_list = [ qt ]
	for i in range(1, n * 2 + 1):
		q_der_list.append(diff(q(x, t), x, i))
	target_eq = qt * I
	for index, value in enumerate(q_der_list):
		if index == 0:
			continue
		key = "a{}".format(index)
		target_eq += coeff_list[key] * value * ( I if (index % 2 == 1) else 1 )
	return { "target_eq" : target_eq, "params" : coeff_list, "q" : q, "y" : y, "x" : x, "t" : t, "q_der_list" : q_der_list, "n" : n }


In [5]:

def f_subs_trav_wave(target_eq = None, params = { }, q = None, y = None, x = None, t = None, q_der_list = [ ], n = None):
	C0 = Symbol("C0")
	k = Symbol("k")
	omega = Symbol("omega")
	b = Symbol("b")
	z = Symbol("z")
	tmpp = [ "C0", "k", "omega", "b", "z" ]
	res_d = dict(zip(tmpp, map(eval, tmpp)))
	subs_fun = y(x - C0 * t) * exp( I * ( k * x - omega * t ) )
	res_d["subs_fun"] = subs_fun
	tmp1 = target_eq.subs({ q(x, t) : subs_fun }).doit().simplify().doit().expand().subs({ x - C0 * t : x }).doit()
	res_d["tmp1"] = tmp1
	tmp2 = tmp1.subs({ exp(I*k*x) : 1, exp(-I*omega*t) : 1}).doit()
	tmp2 = tmp2 - b * y(x)**3
	tmp2 = tmp2.subs({ x : z }).doit()
	res_d["tmp2"] = tmp2
	re_eq = tmp2.subs({ I : 0 }).doit()
	im_eq = tmp2.coeff(I)
	res_d["re_eq"] = re_eq
	res_d["im_eq"] = im_eq
	qq_re = [ i for i in re_eq.as_terms()[-1] if not i.is_Atom ]
	qq_im = [ i for i in im_eq.as_terms()[-1] if not i.is_Atom ]
	res_d["qq_re"] = qq_re
	res_d["qq_im"] = qq_im
	im_eq_coeff = [ im_eq.coeff(i) for i in qq_im ]
	im_eq_sys = [ Eq(i) for i in im_eq_coeff ]
	res_d["im_eq_coeff"] = im_eq_coeff
	res_d["im_eq_sys"] = im_eq_sys
	tt = [ [j for j in i.as_expr().as_ordered_terms()[0].atoms() if j.is_symbol ][0] for i in im_eq_coeff ]
	tt = list(reversed(tt))
	eq_num_list = list(reversed(range(0, len(im_eq_sys))))
	subs_im_d = f_extract_from_eq_sys_(im_eq_sys, tt, eq_num_list)
	res_d["tt"] = tt
	res_d["subs_im_d"] = subs_im_d
	res_d["n"] = n
	return res_d


In [6]:

def f_generate_y_subs(n, a, limit = 4):
	N = Symbol("N")
	A = Symbol("A")
	Xi = Symbol("Xi")
	a = Symbol("a")
	alpha = Symbol("alpha")
	C = Symbol("C")
	R = Function("R", nargs = 1)
	subs_ = { }
	res_subs_ = { }
	subs_[diff(R(z), z, 1)**2] =  R(z)**2 * (1 - Xi * R(z)**2)
	subs_[diff(R(z),z, 2)] = R(z) - 2 * Xi * R(z)**3
	y_subs = a * R(z)**n
	y_subs = A * R(z)**n
	tmp_list = [ y_subs ]
	tmp_keys_list = [ y(z) ]
	for i in range(0, limit):
		tmp_ = tmp_list[i]
		tmp_ = diff(tmp_, z, 1)
		tmp_list.append(tmp_.subs(subs_).simplify().doit())
		tmp_key = tmp_keys_list[i]
		tmp_keys_list.append(diff(tmp_key, z, 1))
	tmp_keys_list = reversed(tmp_keys_list)
	tmp_list = reversed(tmp_list)
	return dict(zip(tmp_keys_list, tmp_list))


In [26]:

q = Function("q", nargs = 2)
y = Function("y", nargs = 1)
x = Symbol("x", real = True)
t = Symbol("t", real = True)


a1, a2, a3, a4 = symbols("a1 a2 a3 a4")

qt = diff(q(x, t), t, 1)
qx = diff(q(x, t), x, 1)
qxx = diff(q(x, t), x, 2)
qxxx = diff(q(x, t), x, 3)
qxxxx = diff(q(x, t), x, 4)



C0 = Symbol("C0")
k = Symbol("k")
omega = Symbol("omega")
b = Symbol("b")
z = Symbol("z")

N = Symbol("N")
A = Symbol("A")
Xi = Symbol("Xi")
a = Symbol("a")
alpha = Symbol("alpha")
C = Symbol("C")
R = Function("R", nargs = 1)

subs_fun = y(x - C0 * t) * exp( I * ( k * x - omega * t ) )

R_subs_ = 2 * exp(z) * C / ( C + Xi * exp(2 * z) )
R_subs_ = R_subs_.subs({ C : 1 })


In [23]:

def f_subs_R(re_eq = None, subs_im_d = { }, qq_re = [ ], n = None, **kwargs):
	res_d = { }
	nn = qq_re[-1].derivative_count
	subs_d = f_generate_y_subs(n, A, limit = nn)
	subs_d_re = dict([ [i, v] for i, v in subs_d.items() if i in qq_re ])
	subs_d_next = { diff(R(z), z, 1)**2 : R(z)**2 * (1 - Xi * R(z)**2) }
	w0 = re_eq
	w0 = re_eq.subs(subs_im_d).doit()
	for i, v in subs_d_re.items():
		w0 = w0.subs({ i : v }).expand().simplify().doit()
	w1 = w0.subs(subs_d_next).doit()
	w2 = w1.expand().simplify().doit()

	h0 = Poly(w2, R(z))

	r_coeffs = h0.coeffs()
	r_eqs = [ Eq(i) for i in r_coeffs ]
	res_d = { "nn" : nn, "subs_d" : subs_d, "subs_d_re" : subs_d_re,
		"subs_d_next" : subs_d_next, "w0" : w0, "w1" : w1, "w2" : w2,
		"h0" : h0, "r_coeffs" : r_coeffs, "r_eqs" : r_eqs }
	return res_d


In [11]:

target_eq = I * qt - I * a1 * qx + a2 * qxx - I * a3 * qxxx + a4 * qxxxx
target_eq = I * qt + I * a1 * qx + a2 * qxx + I * a3 * qxxx + a4 * qxxxx


In [12]:
tmp1 = target_eq.subs({ q(x, t) : subs_fun }).doit().simplify().doit().expand().subs({ x - C0 * t : x }).doit()

In [13]:
tmp1

-I*C0*exp(I*k*x)*exp(-I*omega*t)*Derivative(y(x), x) - a1*k*exp(I*k*x)*exp(-I*omega*t)*y(x) + I*a1*exp(I*k*x)*exp(-I*omega*t)*Derivative(y(x), x) - a2*k**2*exp(I*k*x)*exp(-I*omega*t)*y(x) + 2*I*a2*k*exp(I*k*x)*exp(-I*omega*t)*Derivative(y(x), x) + a2*exp(I*k*x)*exp(-I*omega*t)*Derivative(y(x), (x, 2)) + a3*k**3*exp(I*k*x)*exp(-I*omega*t)*y(x) - 3*I*a3*k**2*exp(I*k*x)*exp(-I*omega*t)*Derivative(y(x), x) - 3*a3*k*exp(I*k*x)*exp(-I*omega*t)*Derivative(y(x), (x, 2)) + I*a3*exp(I*k*x)*exp(-I*omega*t)*Derivative(y(x), (x, 3)) + a4*k**4*exp(I*k*x)*exp(-I*omega*t)*y(x) - 4*I*a4*k**3*exp(I*k*x)*exp(-I*omega*t)*Derivative(y(x), x) - 6*a4*k**2*exp(I*k*x)*exp(-I*omega*t)*Derivative(y(x), (x, 2)) + 4*I*a4*k*exp(I*k*x)*exp(-I*omega*t)*Derivative(y(x), (x, 3)) + a4*exp(I*k*x)*exp(-I*omega*t)*Derivative(y(x), (x, 4)) + omega*exp(I*k*x)*exp(-I*omega*t)*y(x)

In [14]:
res_ = f_gen_target_eq(2)

In [15]:
res_

{'target_eq': I*a1*Derivative(q(x, t), x) + a2*Derivative(q(x, t), (x, 2)) + I*a3*Derivative(q(x, t), (x, 3)) + a4*Derivative(q(x, t), (x, 4)) + I*Derivative(q(x, t), t),
 'params': {'a1': a1, 'a2': a2, 'a3': a3, 'a4': a4},
 'q': q,
 'y': y,
 'x': x,
 't': t,
 'q_der_list': [Derivative(q(x, t), t),
  Derivative(q(x, t), x),
  Derivative(q(x, t), (x, 2)),
  Derivative(q(x, t), (x, 3)),
  Derivative(q(x, t), (x, 4))],
 'n': 2}

In [16]:
f_subs_trav_wave(**res_)

{'C0': C0,
 'k': k,
 'omega': omega,
 'b': b,
 'z': z,
 'subs_fun': exp(I*(k*x - omega*t))*y(-C0*t + x),
 'tmp1': -I*C0*exp(I*k*x)*exp(-I*omega*t)*Derivative(y(x), x) - a1*k*exp(I*k*x)*exp(-I*omega*t)*y(x) + I*a1*exp(I*k*x)*exp(-I*omega*t)*Derivative(y(x), x) - a2*k**2*exp(I*k*x)*exp(-I*omega*t)*y(x) + 2*I*a2*k*exp(I*k*x)*exp(-I*omega*t)*Derivative(y(x), x) + a2*exp(I*k*x)*exp(-I*omega*t)*Derivative(y(x), (x, 2)) + a3*k**3*exp(I*k*x)*exp(-I*omega*t)*y(x) - 3*I*a3*k**2*exp(I*k*x)*exp(-I*omega*t)*Derivative(y(x), x) - 3*a3*k*exp(I*k*x)*exp(-I*omega*t)*Derivative(y(x), (x, 2)) + I*a3*exp(I*k*x)*exp(-I*omega*t)*Derivative(y(x), (x, 3)) + a4*k**4*exp(I*k*x)*exp(-I*omega*t)*y(x) - 4*I*a4*k**3*exp(I*k*x)*exp(-I*omega*t)*Derivative(y(x), x) - 6*a4*k**2*exp(I*k*x)*exp(-I*omega*t)*Derivative(y(x), (x, 2)) + 4*I*a4*k*exp(I*k*x)*exp(-I*omega*t)*Derivative(y(x), (x, 3)) + a4*exp(I*k*x)*exp(-I*omega*t)*Derivative(y(x), (x, 4)) + omega*exp(I*k*x)*exp(-I*omega*t)*y(x),
 'tmp2': -I*C0*Derivative(y(z), 

In [17]:
res_ = f_gen_target_eq(3)

In [18]:
res_

{'target_eq': I*a1*Derivative(q(x, t), x) + a2*Derivative(q(x, t), (x, 2)) + I*a3*Derivative(q(x, t), (x, 3)) + a4*Derivative(q(x, t), (x, 4)) + I*a5*Derivative(q(x, t), (x, 5)) + a6*Derivative(q(x, t), (x, 6)) + I*Derivative(q(x, t), t),
 'params': {'a1': a1, 'a2': a2, 'a3': a3, 'a4': a4, 'a5': a5, 'a6': a6},
 'q': q,
 'y': y,
 'x': x,
 't': t,
 'q_der_list': [Derivative(q(x, t), t),
  Derivative(q(x, t), x),
  Derivative(q(x, t), (x, 2)),
  Derivative(q(x, t), (x, 3)),
  Derivative(q(x, t), (x, 4)),
  Derivative(q(x, t), (x, 5)),
  Derivative(q(x, t), (x, 6))],
 'n': 3}

In [21]:
tt = f_subs_trav_wave(**res_)
tt

{'C0': C0,
 'k': k,
 'omega': omega,
 'b': b,
 'z': z,
 'subs_fun': exp(I*(k*x - omega*t))*y(-C0*t + x),
 'tmp1': -I*C0*exp(I*k*x)*exp(-I*omega*t)*Derivative(y(x), x) - a1*k*exp(I*k*x)*exp(-I*omega*t)*y(x) + I*a1*exp(I*k*x)*exp(-I*omega*t)*Derivative(y(x), x) - a2*k**2*exp(I*k*x)*exp(-I*omega*t)*y(x) + 2*I*a2*k*exp(I*k*x)*exp(-I*omega*t)*Derivative(y(x), x) + a2*exp(I*k*x)*exp(-I*omega*t)*Derivative(y(x), (x, 2)) + a3*k**3*exp(I*k*x)*exp(-I*omega*t)*y(x) - 3*I*a3*k**2*exp(I*k*x)*exp(-I*omega*t)*Derivative(y(x), x) - 3*a3*k*exp(I*k*x)*exp(-I*omega*t)*Derivative(y(x), (x, 2)) + I*a3*exp(I*k*x)*exp(-I*omega*t)*Derivative(y(x), (x, 3)) + a4*k**4*exp(I*k*x)*exp(-I*omega*t)*y(x) - 4*I*a4*k**3*exp(I*k*x)*exp(-I*omega*t)*Derivative(y(x), x) - 6*a4*k**2*exp(I*k*x)*exp(-I*omega*t)*Derivative(y(x), (x, 2)) + 4*I*a4*k*exp(I*k*x)*exp(-I*omega*t)*Derivative(y(x), (x, 3)) + a4*exp(I*k*x)*exp(-I*omega*t)*Derivative(y(x), (x, 4)) - a5*k**5*exp(I*k*x)*exp(-I*omega*t)*y(x) + 5*I*a5*k**4*exp(I*k*x)*exp(-I

In [28]:
tt1 = f_subs_R(**tt)
tt1

{'nn': 6,
 'subs_d': {Derivative(y(z), (z, 6)): 3*A*(-3360*Xi**3*R(z)**6 + 5240*Xi**2*R(z)**4 + 3360*Xi**2*R(z)**2*Derivative(R(z), z)**2 - 2364*Xi*R(z)**2 - 1360*Xi*Derivative(R(z), z)**2 + 243)*R(z)**3,
  Derivative(y(z), (z, 5)): 3*A*(840*Xi**2*R(z)**4 - 680*Xi*R(z)**2 + 81)*R(z)**2*Derivative(R(z), z),
  Derivative(y(z), (z, 4)): 3*A*(120*Xi**2*R(z)**4 - 136*Xi*R(z)**2 + 27)*R(z)**3,
  Derivative(y(z), (z, 3)): 3*A*(-20*Xi*R(z)**2 + 9)*R(z)**2*Derivative(R(z), z),
  Derivative(y(z), (z, 2)): 3*A*(-4*Xi*R(z)**2 + 3)*R(z)**3,
  Derivative(y(z), z): 3*A*R(z)**2*Derivative(R(z), z),
  y(z): A*R(z)**3},
 'subs_d_re': {Derivative(y(z), (z, 6)): 3*A*(-3360*Xi**3*R(z)**6 + 5240*Xi**2*R(z)**4 + 3360*Xi**2*R(z)**2*Derivative(R(z), z)**2 - 2364*Xi*R(z)**2 - 1360*Xi*Derivative(R(z), z)**2 + 243)*R(z)**3,
  Derivative(y(z), (z, 4)): 3*A*(120*Xi**2*R(z)**4 - 136*Xi*R(z)**2 + 27)*R(z)**3,
  Derivative(y(z), (z, 2)): 3*A*(-4*Xi*R(z)**2 + 3)*R(z)**3,
  y(z): A*R(z)**3},
 'subs_d_next': {Derivative(

In [29]:
r_eqs = tt1["r_eqs"]

In [30]:
r_eqs

[Eq(-A**3*b - 20160*A*Xi**3*a6, 0),
 Eq(360*A*Xi**2*a4 + 5400*A*Xi**2*a6*k**2 + 29880*A*Xi**2*a6, 0),
 Eq(-12*A*Xi*a2 - 72*A*Xi*a4*k**2 - 408*A*Xi*a4 - 900*A*Xi*a6*k**4 - 6120*A*Xi*a6*k**2 - 11172*A*Xi*a6, 0),
 Eq(-A*a1*k - A*a2*k**2 + 9*A*a2 - 3*A*a4*k**4 + 54*A*a4*k**2 + 81*A*a4 - 35*A*a6*k**6 + 675*A*a6*k**4 + 1215*A*a6*k**2 + 729*A*a6 + A*omega, 0)]

In [121]:
subs_re_d = f_extract_from_eq_sys_(r_eqs, [ b, a4, Symbol("a2"), omega ], [ 0, 1, 2, 3 ])

In [122]:
tmp2 = tt["tmp2"]
target_eq_y = tmp2
tmp2

-I*C0*Derivative(y(z), z) - a1*k*y(z) + I*a1*Derivative(y(z), z) - a2*k**2*y(z) + 2*I*a2*k*Derivative(y(z), z) + a2*Derivative(y(z), (z, 2)) + a3*k**3*y(z) - 3*I*a3*k**2*Derivative(y(z), z) - 3*a3*k*Derivative(y(z), (z, 2)) + I*a3*Derivative(y(z), (z, 3)) + a4*k**4*y(z) - 4*I*a4*k**3*Derivative(y(z), z) - 6*a4*k**2*Derivative(y(z), (z, 2)) + 4*I*a4*k*Derivative(y(z), (z, 3)) + a4*Derivative(y(z), (z, 4)) - a5*k**5*y(z) + 5*I*a5*k**4*Derivative(y(z), z) + 10*a5*k**3*Derivative(y(z), (z, 2)) - 10*I*a5*k**2*Derivative(y(z), (z, 3)) - 5*a5*k*Derivative(y(z), (z, 4)) + I*a5*Derivative(y(z), (z, 5)) - a6*k**6*y(z) + 6*I*a6*k**5*Derivative(y(z), z) + 15*a6*k**4*Derivative(y(z), (z, 2)) - 20*I*a6*k**3*Derivative(y(z), (z, 3)) - 15*a6*k**2*Derivative(y(z), (z, 4)) + 6*I*a6*k*Derivative(y(z), (z, 5)) + a6*Derivative(y(z), (z, 6)) - b*y(z)**3 + omega*y(z)

In [123]:
subs_d = tt1["subs_d"]
target_eq_y_subs = subs_d[y(z)]

In [124]:
subs_im_d = tt["subs_im_d"]

In [125]:
all_coeff_subs_d = { }
all_coeff_subs_d.update(subs_re_d)
all_coeff_subs_d.update(subs_im_d)

In [126]:
target_eq_y_ = target_eq_y.subs(all_coeff_subs_d).expand().simplify().doit()

In [127]:
TMP0 = target_eq_y_.subs({ y(z) : target_eq_y_subs }).doit()

In [128]:
TMP1 = TMP0.subs({ R(z) : R_subs_ }).simplify().factor().simplify().doit()

In [129]:
TMP1.simplify()

0

In [117]:

def f_line_up_one_param(eq, not_include = [ ]):
		f_symb = eq.lhs.free_symbols
		a_symb = [ i for i in f_symb if "a" == i.name[0] and i not in not_include ]
		pass
		d_ = dict([ [ i, sp.degree(eq, i) ] for i in a_symb ])
		line_up_param = min(d_, key = lambda x : d_[x])
		d_min_ = [ ]
		for key in d_.keys():
				if d_[key] == d_[line_up_param]:
						d_min_.append(key)
		line_up_param = min(d_min_, key = lambda x : int(x.name[1:]))
		return { "f_symb" : f_symb, "a_symb" : a_symb, "d_" : d_, "d_min_" : d_min_, "line_up_param" : line_up_param }


In [118]:

def f_line_up_params(eqs):
		pass
		line_up_params_list = [ ]
		new_eqs = eqs[ 1 : -1 ]
		for eq in new_eqs:
				tmp_ = f_line_up_one_param(eq, not_include = line_up_params_list)
				line_up_params_list.append(tmp_["line_up_param"])
		return line_up_params_list


In [119]:

f_line_up_params(r_eqs)



[a4, a2]

In [120]:
f_line_up_one_param(r_eqs[3])

{'f_symb': {A, a1, a2, a4, a6, k, omega},
 'a_symb': [a4, a6, a2, a1],
 'd_': {a4: 1, a6: 1, a2: 1, a1: 1},
 'd_min_': [a4, a6, a2, a1],
 'line_up_param': a1}

In [71]:
subs_re_d

{b: -20160*Xi**3*a6/A**2,
 a4: -a6*(15*k**2 + 83),
 a2: a6*(15*k**4 + 498*k**2 + 1891),
 omega: a1*k + 5*a6*k**6 + 249*a6*k**4 + 1891*a6*k**2 - 11025*a6}